In [44]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import pandas as pd


In [45]:
data =  pd.read_csv("melb_data.csv")
data.head()


,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [46]:
#remove redundant and irrelevant attributes:
#Address has 13378 unique values, almost unique for each object. Description rather than category, also overlaps with latitude, longitude, regionname
#method simply describes if property was put on auction or sold
data['Address'].value_counts()

36 Aberfeldie St    3
2 Bruce St          3
5 Charles St        3
53 William St       3
14 Arthur St        3
                   ..
16 Alleford St      1
2/1073 Centre Rd    1
14 Columbia St      1
21 Hardy Ct         1
6 Agnes St          1
Name: Address, Length: 13378, dtype: int64

In [47]:
#remove redundant and irrelevant attributes:
#postcode is redundant
data['Postcode'].value_counts()

3073.0    359
3020.0    306
3121.0    292
3040.0    290
3046.0    284
         ... 
3340.0      1
3158.0      1
3808.0      1
3777.0      1
3793.0      1
Name: Postcode, Length: 198, dtype: int64

In [48]:
#remove redundant and irrelevant attributes:
#method simply describes how the data is obtained (sold, auction), is irrelevant
data = data.drop(columns=['Address','Method','Postcode'])

In [49]:
#data cleaning: checking for duplicates
data.duplicated().sum()

0

In [50]:
#data cleaning: missing values
data.isna().sum()
#missing values for car can be removed as very few, buildingarea, yearbuilt, councilarea needs replacement

Suburb              0
Rooms               0
Type                0
Price               0
SellerG             0
Date                0
Distance            0
Bedroom2            0
Bathroom            0
Car                62
Landsize            0
BuildingArea     6450
YearBuilt        5375
CouncilArea      1369
Lattitude           0
Longtitude          0
Regionname          0
Propertycount       0
dtype: int64

In [51]:
#data cleaning: missing values
data = data.dropna(subset=['Car'])
data.reset_index(drop=True, inplace=True)
data['BuildingArea'] = data['BuildingArea'].fillna(data['BuildingArea'].median())
data['YearBuilt'] = data['YearBuilt'].fillna(data['YearBuilt'].median())
data['CouncilArea'] = data['CouncilArea'].fillna(data['CouncilArea'].mode()[0])

In [52]:
#data transformation: yearbuilt to age
data['BuildingAge'] = 2018-data['YearBuilt']
data = data.drop(columns=['YearBuilt'])

In [53]:
#creating features from date sold
data['Date'] = pd.to_datetime(data['Date'])
data['Year_Sold'] = data['Date'].dt.year
data['Month_Sold'] = data['Date'].dt.month
data['Day_Sold'] = data['Date'].dt.day
data['DayOfWeek_Sold'] = data['Date'].dt.dayofweek
data.drop(columns=['Date'])
data[['Year_Sold', 'Month_Sold', 'Day_Sold', 'DayOfWeek_Sold']].head()

C:\Users\shiyi\AppData\Local\Temp\ipykernel_17008\2262746614.py:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  data['Date'] = pd.to_datetime(data['Date'])


,Year_Sold,Month_Sold,Day_Sold,DayOfWeek_Sold
0,2016,3,12,5
1,2016,4,2,5
2,2017,4,3,0
3,2017,4,3,0
4,2016,4,6,2


In [54]:
#encoding categorical attributes
#suburb, SellerG, CouncilArea are nominal, have high cardinality - binary encoder used for memory concerns
#one-hot encoder for other nominal features - better performance (https://www.diva-portal.org/smash/get/diva2:1259073/FULLTEXT01.pdf)
categorical = ['Suburb','Type','SellerG','CouncilArea','Regionname']
for x in categorical:
    print(data[x].value_counts())

Reservoir         359
Richmond          259
Bentleigh East    249
Preston           239
Brunswick         221
                 ... 
Sandhurst           1
Bullengarook        1
Montrose            1
Croydon South       1
Monbulk             1
Name: Suburb, Length: 314, dtype: int64
h    9389
u    3015
t    1114
Name: Type, dtype: int64
Nelson           1558
Jellis           1303
hockingstuart    1160
Barry            1010
Ray               701
                 ... 
Prowse              1
Luxe                1
Zahn                1
Homes               1
Point               1
Name: SellerG, Length: 268, dtype: int64
Moreland             2470
Boroondara           1160
Moonee Valley         997
Darebin               934
Glen Eira             848
Stonnington           719
Maribyrnong           692
Yarra                 647
Port Phillip          628
Banyule               594
Bayside               489
Melbourne             470
Hobsons Bay           434
Brimbank              424
Monash          

In [55]:
#encoding categorical attributes
from category_encoders.binary import BinaryEncoder
encoder = BinaryEncoder(cols=['Suburb','SellerG','CouncilArea'], return_df=True)
data_encoded = encoder.fit_transform(data)
type_encoded = pd.get_dummies(data['Type'], prefix='Type')
regionname_encoded = pd.get_dummies(data['Regionname'], prefix='Regionname')
data_encoded = pd.concat([data_encoded, type_encoded, regionname_encoded], axis='columns').drop(['Type','Regionname'],axis=1)
data_encoded.columns

Index(['Suburb_0', 'Suburb_1', 'Suburb_2', 'Suburb_3', 'Suburb_4', 'Suburb_5',
       'Suburb_6', 'Suburb_7', 'Suburb_8', 'Rooms', 'Price', 'SellerG_0',
       'SellerG_1', 'SellerG_2', 'SellerG_3', 'SellerG_4', 'SellerG_5',
       'SellerG_6', 'SellerG_7', 'SellerG_8', 'Date', 'Distance', 'Bedroom2',
       'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'CouncilArea_0',
       'CouncilArea_1', 'CouncilArea_2', 'CouncilArea_3', 'CouncilArea_4',
       'CouncilArea_5', 'Lattitude', 'Longtitude', 'Propertycount',
       'BuildingAge', 'Year_Sold', 'Month_Sold', 'Day_Sold', 'DayOfWeek_Sold',
       'Type_h', 'Type_t', 'Type_u', 'Regionname_Eastern Metropolitan',
       'Regionname_Eastern Victoria', 'Regionname_Northern Metropolitan',
       'Regionname_Northern Victoria', 'Regionname_South-Eastern Metropolitan',
       'Regionname_Southern Metropolitan', 'Regionname_Western Metropolitan',
       'Regionname_Western Victoria'],
      dtype='object')

In [56]:
#data transformation: standardization, not normalization, as data contains many outliers
data.boxplot()
plt.yscale("log")
plt.xticks(rotation=90)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16]),
 [Text(1, 0, 'Rooms'),
  Text(2, 0, 'Price'),
  Text(3, 0, 'Distance'),
  Text(4, 0, 'Bedroom2'),
  Text(5, 0, 'Bathroom'),
  Text(6, 0, 'Car'),
  Text(7, 0, 'Landsize'),
  Text(8, 0, 'BuildingArea'),
  Text(9, 0, 'Lattitude'),
  Text(10, 0, 'Longtitude'),
  Text(11, 0, 'Propertycount'),
  Text(12, 0, 'BuildingAge'),
  Text(13, 0, 'Year_Sold'),
  Text(14, 0, 'Month_Sold'),
  Text(15, 0, 'Day_Sold'),
  Text(16, 0, 'DayOfWeek_Sold')])

In [57]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data_scaled = data_encoded.copy()
numeric_columns = ['Rooms','Distance','Bedroom2','Bathroom','Car','Landsize','BuildingArea','BuildingAge','Lattitude','Longtitude','Propertycount']
data_scaled[numeric_columns] = scaler.fit_transform(data_scaled[numeric_columns])
data_scaled

,Suburb_0,Suburb_1,Suburb_2,Suburb_3,Suburb_4,Suburb_5,Suburb_6,Suburb_7,Suburb_8,Rooms,...,Type_t,Type_u,Regionname_Eastern Metropolitan,Regionname_Eastern Victoria,Regionname_Northern Metropolitan,Regionname_Northern Victoria,Regionname_South-Eastern Metropolitan,Regionname_Southern Metropolitan,Regionname_Western Metropolitan,Regionname_Western Victoria
0,0,0,0,0,0,0,0,0,1,-0.982624,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,-0.982624,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0.062961,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0.062961,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,1.108546,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13513,0,1,1,1,1,1,0,0,1,1.108546,...,0,0,0,0,0,0,1,0,0,0
13514,0,0,1,1,0,1,0,0,1,0.062961,...,0,0,0,0,0,0,0,0,1,0
13515,0,0,1,1,0,1,0,0,1,0.062961,...,0,0,0,0,0,0,0,0,1,0
13516,0,0,1,1,0,1,0,0,1,1.108546,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
#pca
